# Goal text analysis - NB
# Bag of Words (BoW)

In [1]:
import psycopg2
import pandas as pd
import numpy as np
import seaborn as sns
import statsmodels.api as sm
import random
import re
import matplotlib.pyplot as plt
%matplotlib inline

/anaconda/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
conn_laurel = psycopg2.connect("dbname='analytics' user='analytics' host='analytics.cv90snkxh2gd.us-west-2.rds.amazonaws.com' password='!TgP$Ol9Z&6QhKW0tmn9mOW5rYT2J8'")
conn_hardy = psycopg2.connect("dbname='analytics' user='u_yidong' host='udacity-segment.c2zpsqalam7o.us-west-2.redshift.amazonaws.com' port='5439' password='Lyd580809?!'")

In [3]:
sql_apps = "SELECT distinct a.id as application_id,a.cohort_id,a.applicant_id,a.nd_key,a.application_type\
      ,coalesce(a.applicant_country,'Unknown') as applicant_country\
      ,coalesce(a.applicant_geo,'Unknown') as applicant_geo\
      ,case when b.user_id is not null then 'term completed'\
            when a.first_charge_created_at is not null and a.accepted_at is not null then 'paid'\
            when a.accepted_at is not null then 'accepted'\
            when a.rejected_at is not null then 'rejected'\
            when a.submitted_at is not null then 'submitted'\
       else 'applied' end as application_status\
      ,current_date - a.applicant_created_at::date as user_age\
      ,a.cohort_notify_at::date - a.cohort_open_at::date as cohort_open_to_notify\
      ,a.cohort_close_at::date - a.cohort_open_at::date as cohort_open_to_close\
      ,date_part('month',a.cohort_open_at) as cohort_open_month\
      ,date_part('month',a.application_created_at) as application_month\
      ,a.application_created_at::date - a.cohort_open_at::date as apply_before_start\
      ,a.submitted_at::date - a.application_created_at::date as apply_to_submit\
      ,coalesce(a.accepted_at,a.rejected_at)::date - a.submitted_at::date as submit_to_decision\
      ,a.first_charge_created_at::date - a.accepted_at::date as accepted_to_pay\
      ,coalesce(a.price,0)::int as price\
       FROM admissions.applications as a\
       LEFT JOIN enrollments.term_completions as b ON a.applicant_id = b.user_id and a.nd_key = b.nd_key\
       WHERE accepted_at is not null and first_charge_created_at is not null\
         AND email not like '%@udacity%';"

df_apps = pd.read_sql(sql_apps,conn_laurel)
df_apps.head()

,application_id,cohort_id,applicant_id,nd_key,application_type,applicant_country,applicant_geo,application_status,user_age,cohort_open_to_notify,cohort_open_to_close,cohort_open_month,application_month,apply_before_start,apply_to_submit,submit_to_decision,accepted_to_pay,price
0,31cdc80b-885f-4623-ba75-9b968666c59e,18,10517481523,nd013,term,US,US,paid,231,88,70,2.0,3.0,35,NaN,NaN,1,800
1,fc153d34-bbd8-435a-8139-e66c32ba70ef,88,10738132589,nd013,term,DE,EMEA,paid,123,70,56,6.0,7.0,17,3.0,51.0,5,800
2,9d9feb78-c936-4047-8f9a-7fc556fd7fed,18,10588451917,nd013,term,CZ,EMEA,term completed,200,88,70,2.0,4.0,66,0.0,20.0,2,800
3,5562ce8c-06d4-4dbf-a2f9-4f6eaaee2d19,283,10923306031,nd889,term,US,US,paid,42,71,62,9.0,9.0,6,34.0,4.0,0,800
4,303dc4a8-f907-4075-83bb-cce3675a04bf,18,10436444018,nd013,term,US,US,term completed,263,88,70,2.0,2.0,3,0.0,83.0,5,800


In [4]:
sql_course = "select application_id,cohort_id,user_id AS applicant_id,nanodegree_key AS nd_key\
       ,count(distinct course_key) as num_courses\
       ,count(distinct case when course_finished = 2 then course_key else null end) as num_course_finished\
       ,count(distinct case when course_finished = 1 then course_key else null end) as num_course_enrolled\
       ,current_date - first_course_started_at::date AS user_study_age\
 from (select distinct b.application_id,b.cohort_id,a.user_id,b.nanodegree_key,course_key,join_time,leave_time\
             ,first_value(join_time) over (partition by a.user_id order by join_time rows between unbounded preceding and unbounded following) as first_course_started_at\
             ,case when a.leave_time::Date <= b.received_at::Date then 2\
                   when a.leave_time is not null then 1 \
              else 0 end as course_finished\
       from analytics_tables.course_enrollments as a\
       inner join backend_admissions.application_received as b on a.user_id = b.user_id\
       where a.join_time <= b.received_at\
      ) group by application_id,cohort_id,user_id,nanodegree_key,current_date - first_course_started_at::date"

df_course = pd.read_sql(sql_course,conn_hardy)
df_course.head()

,application_id,cohort_id,applicant_id,nd_key,num_courses,num_course_finished,num_course_enrolled,user_study_age
0,c800f6d5-65cc-47bf-afac-832b89901cef,eu-google-sep-2017,10005272616,google-dev-scholarship,3,0,0,70
1,9cb0245f-63d0-48f9-a8fb-8c139032243e,111,10007824085,nd889,7,0,0,329
2,090429c1-252b-47db-85b2-5b0400c5f72e,eu-google-sep-2017,10017580226,google-dev-scholarship,2,0,0,360
3,7e231e1b-7689-4166-a23d-a03b28661a0c,220,10022232224,nd013,8,0,0,357
4,ec56d7bb-3e07-494a-951c-bd3e60a00eab,eu-google-sep-2017,10025113006,google-dev-scholarship,2,0,0,356


In [5]:
df = pd.merge(df_apps,df_course,on=['application_id','cohort_id','applicant_id','nd_key'],how='inner')
df.head()

,application_id,cohort_id,applicant_id,nd_key,application_type,applicant_country,applicant_geo,application_status,user_age,cohort_open_to_notify,...,application_month,apply_before_start,apply_to_submit,submit_to_decision,accepted_to_pay,price,num_courses,num_course_finished,num_course_enrolled,user_study_age
0,612e53be-80ef-4bdb-8121-6e8b3621bb5d,18,5199658542,nd013,term,US,US,paid,203,88,...,4.0,63,6.0,17.0,8,800,5,0,1,775
1,6820fa4f-0448-4be3-b46d-c6330c83f2a2,47,6603677391,nd013,term,KR,APAC,paid,181,65,...,5.0,15,0.0,50.0,43,800,2,0,0,652
2,8a0c7fad-1580-40f5-97f5-a46db9a24509,89,4153458585,nd013,term,US,US,paid,264,43,...,9.0,19,8.0,17.0,3,800,27,5,0,934
3,b287a0a0-f735-4bd7-bc3f-285ddb7b0c05,100,5478460939,nd209,term,US,US,paid,186,92,...,5.0,2,0.0,89.0,7,1200,2,0,0,731
4,9a43a21a-601a-4e05-80f1-d602d23c1ddf,18,9930098687,nd013,term,US,US,term completed,212,88,...,4.0,54,0.0,32.0,9,800,2,0,0,373


In [6]:
# paid students only
#nd013 and nd209 has most examples
data = df.loc[df['application_status'].isin(['paid','term completed'])]
data['application_status'] = data['application_status'].map({'paid':0,'term completed':1})

In [7]:
# questions
sql_questions = "select distinct application_id,cohort_id,applicant_id,nd_key\
                        ,question_id,question_prompt,response,question_prompt||' A:'||response as q_r\
                from admissions.question_responses\
                where application_type != 'scholarship' and response is not null\
                  and question_id in ('question_id','48e7b492-62b4-4d99-b596-80d68f2966ae','cad2565e-8176-44b5-9080-14cfc67ea9b4','fba3666b-db04-46e9-8f3d-2a303f13e0a5','26330483-09a3-444a-b534-e9cd7021ec1c','42b96348-56c4-4e3b-b0f9-8d3911969e48','516737fe-4567-48d2-8645-08d335e21b0c','0c22dbaa-bf1d-4c67-be34-a61014591d6f','2ad03aaa-1b35-4381-9c43-907b1b4eba67','6afe0061-746b-4bd7-807c-393fe5c7599d','779c3b6c-3648-423b-8d3f-8a4f36f23e2a','1346c396-a687-4082-a10e-d3556ac50b4b','1cfe0061-746b-4bd7-807c-393fe5c7599d','ed06fc88-6a59-40a8-b7ea-ece90d83c333','fa332682-b62e-4791-b95f-c528b532a754','e42e1049-e459-4602-ae4b-cb78098dcf89','01f28833-152f-438d-a831-51e33583bde5','7c55f3cc-1c0b-4d95-97f9-cc3409e3145f','a633a338-9726-472a-a0ff-0d8695b30638','f1a93ec7-e2cd-46b6-ba28-2f36b30539a9','6967091c-09c6-4455-9f1e-d0de318bacc5','011ef02c-abea-476d-afdc-d1aa8a456b94','0f617540-5680-480c-aaf3-0663c17c3c7d','1dfe0061-746b-4bd7-807c-393fe5c7599d','20f21bd0-eb3e-4898-ae1f-eb994f9ce823','45781b84-093b-41c7-b5a4-c521f343bf21','47d35057-b1fb-4106-8218-7e551497420e','491b500b-26a2-425b-b142-00773f1d1938','5cf34496-6bc3-4c8d-9c6a-7c037f48669f','70ffc637-2162-484a-b863-3a1a1469f106','7a09e78a-8000-459e-9d79-aa21db135138','823a8248-b7d7-4dbe-a619-2969ca453f1e','8afe0061-746b-4bd7-807c-393fe5c7599d','8ea6732d-0ef9-489d-bf11-20673183a0c2','9481dc6b-c4e9-4207-a30c-65c3ac302bf5','995331ea-092b-4b71-8122-a0c65214c765','9d5c6390-a690-4d6d-8aaa-c60a772b4550','a26ac91f-c263-49f1-8171-9821bc4d2da5','a71ff6e3-1b33-4e11-94f1-5b94d786b5ab','a7b7f95b-1924-4189-840e-b5c407545908','ae155453-72b2-4247-8da5-dced5270f29b','bdc024d8-7712-48ff-b86e-292be1df0d9f','d3b0e6c7-0601-4651-851f-7ef4bea1aa6d','d8f9089c-c3a5-4d77-9c37-ae248e134e42','d9af077d-b729-4971-84b8-f86f390789c1','da04a859-199c-42cf-ab5d-6132bc529161','dc898f2e-1810-4d93-9db0-f381778d45e1','f254514a-26f4-4d29-83af-711c27d4862d','f34c024a-b446-464e-add0-558c9eb8a3c8','fa537969-017a-4816-a317-b5883305c723')"
df_questions = pd.read_sql(sql_questions,conn_laurel)
df_questions.head()

,application_id,cohort_id,applicant_id,nd_key,question_id,question_prompt,response,q_r
0,0001883b-f88d-4662-afea-717fbba580a0,129,4746120864,nd889,2ad03aaa-1b35-4381-9c43-907b1b4eba67,What do you hope to accomplish through this pr...,I am currently writing Machine Learning Code a...,What do you hope to accomplish through this pr...
1,0001883b-f88d-4662-afea-717fbba580a0,129,4746120864,nd889,48e7b492-62b4-4d99-b596-80d68f2966ae,What is the highest level of education you hav...,Master's degree,What is the highest level of education you hav...
2,0001883b-f88d-4662-afea-717fbba580a0,129,4746120864,nd889,516737fe-4567-48d2-8645-08d335e21b0c,What gender do you most identify with?,Male,What gender do you most identify with? A:Male
3,0001883b-f88d-4662-afea-717fbba580a0,129,4746120864,nd889,6967091c-09c6-4455-9f1e-d0de318bacc5,How many years of professional experience do y...,5+ years,How many years of professional experience do y...
4,0001883b-f88d-4662-afea-717fbba580a0,129,4746120864,nd889,779c3b6c-3648-423b-8d3f-8a4f36f23e2a,What is your primary purpose in taking this pr...,Grow skills for my current role,What is your primary purpose in taking this pr...


In [8]:
cols = ['application_id','cohort_id','applicant_id','nd_key','application_status']
df_qr = pd.merge(data[cols],df_questions,on=['application_id','cohort_id','applicant_id','nd_key'],how='inner')
df_qr.shape

(45555, 9)

In [9]:
#data
df_goal = df_qr.loc[df_questions['question_id'].isin(['2ad03aaa-1b35-4381-9c43-907b1b4eba67','6afe0061-746b-4bd7-807c-393fe5c7599d'])]
#df_goal = df_goal.assign(goal='')

In [10]:
df_goal.groupby(['application_status'])['application_id'].count()

application_status
0    2805
1     466
Name: application_id, dtype: int64

In [11]:
goal = set(df_goal['response'])

In [12]:
#lower case
#df_goal['goal'] = df_goal['response'].apply(lambda x: x.lower())
goal_lower_case = []
for i in goal:
    goal_lower_case.append(i.lower())

In [13]:
#remove punctuation
import string
#df_goal['goal'] = df_goal['goal'].apply(lambda x: x.translate(str.maketrans('','',string.punctuation)))
goal_remove_punctuation = []
for i in goal_lower_case:
    goal_remove_punctuation.append(i.translate(str.maketrans('', '', string.punctuation)))

In [14]:
#tokenization
#df_goal['goal'] = df_goal['goal'].apply(lambda x: x.split(' '))
processed_goal = []
for i in goal_remove_punctuation:
    processed_goal.append(i.split(' '))

In [15]:
#count frequencies
import pprint
from collections import Counter

frequency_list = []

for i in processed_goal:
    frequency_counts = Counter(i)
    frequency_list.append(frequency_counts)
#pprint.pprint(frequency_list)

# impletemnt BoW in scikit-learn

In [16]:
from sklearn.feature_extraction.text import CountVectorizer
count_vector = CountVectorizer()

In [17]:
print(count_vector)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)


In [18]:
count_vector.fit(goal)
count_vector.get_feature_names()

['000',
 '01',
 '022',
 '10',
 '100',
 '1016',
 '11',
 '12',
 '13',
 '133',
 '141',
 '146',
 '15',
 '180',
 '19',
 '1s',
 '20',
 '2000s',
 '2004',
 '2006',
 '2011',
 '2012',
 '2014',
 '2015',
 '2016',
 '2017',
 '2018',
 '2020',
 '25',
 '26',
 '2d',
 '2m2fmty',
 '2mm58il',
 '2nd',
 '30',
 '30k',
 '361',
 '377',
 '3d',
 '40',
 '42',
 '4th',
 '50',
 '500',
 '512',
 '54a3a992',
 '60',
 '70',
 '80',
 '90',
 'abap',
 'abilities',
 'ability',
 'able',
 'about',
 'above',
 'abu',
 'ac',
 'academia',
 'academic',
 'academy',
 'accelerate',
 'accelerators',
 'accenture',
 'accept',
 'accepted',
 'access',
 'accidents',
 'acclaimable',
 'accomplish',
 'accomplished',
 'accomplishment',
 'according',
 'account',
 'accumulate',
 'accuracy',
 'accurately',
 'achieve',
 'achieved',
 'achievement',
 'achievements',
 'achieving',
 'acknowledge',
 'acquainted',
 'acquire',
 'acquired',
 'acquiring',
 'across',
 'acting',
 'action',
 'actions',
 'activated',
 'active',
 'actively',
 'actual',
 'actuators

In [19]:
doc_array = count_vector.transform(goal).toarray()
doc_array

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ..., 
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [20]:
frequency_matrix = pd.DataFrame(doc_array, 
                                columns = count_vector.get_feature_names())
frequency_matrix.head(5)

,000,01,022,10,100,1016,11,12,13,133,...,your,youtu,youtube,ypc,yrs,yuan,yxypwu3zddm,zed,zero,zfs
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# training/test set

In [21]:
from sklearn.cross_validation import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df_goal['response'], 
                                                    df_goal['application_status'], 
                                                    random_state=1)

print('Number of rows in the total set: {}'.format(df_goal.shape[0]))
print('Number of rows in the training set: {}'.format(X_train.shape[0]))
print('Number of rows in the test set: {}'.format(X_test.shape[0]))

Number of rows in the total set: 3271
Number of rows in the training set: 2453
Number of rows in the test set: 818


/anaconda/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [22]:
pd.DataFrame(y_train).groupby('application_status')['application_status'].count()

application_status
0    2095
1     358
Name: application_status, dtype: int64

In [23]:
pd.DataFrame(y_test).groupby('application_status')['application_status'].count()

application_status
0    710
1    108
Name: application_status, dtype: int64

In [24]:
training_data = count_vector.fit_transform(X_train)
testing_data = count_vector.transform(X_test)

In [25]:
from sklearn.naive_bayes import MultinomialNB
naive_bayes = MultinomialNB()
naive_bayes.fit(training_data, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [26]:
predictions = naive_bayes.predict(testing_data)

In [27]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
print('Accuracy score: ', format(accuracy_score(y_test, predictions)))
print('Precision score: ', format(precision_score(y_test, predictions)))
print('Recall score: ', format(recall_score(y_test, predictions)))
print('F1 score: ', format(f1_score(y_test, predictions)))

Accuracy score:  0.8679706601466992
Precision score:  0.0
Recall score:  0.0
F1 score:  0.0


/anaconda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/anaconda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


In [28]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, predictions)

array([[710,   0],
       [108,   0]])

In [29]:
d_1 = df_goal.loc[df_goal['application_status']==1].reset_index()
d_2 = df_goal.loc[df_goal['application_status']==0].reset_index()
n = d_1.shape[0]
d_2_s = d_2.iloc[random.sample(range(d_2.shape[0]),n),:]
d = pd.concat([d_1,d_2_s],axis = 0)
d.groupby(['application_status'])['application_id'].count()

application_status
0    466
1    466
Name: application_id, dtype: int64

In [30]:
X_train, X_test, y_train, y_test = train_test_split(d['response'], 
                                                    d['application_status'], 
                                                    random_state=1)

print('Number of rows in the total set: {}'.format(d.shape[0]))
print('Number of rows in the training set: {}'.format(X_train.shape[0]))
print('Number of rows in the test set: {}'.format(X_test.shape[0]))

Number of rows in the total set: 932
Number of rows in the training set: 699
Number of rows in the test set: 233


In [31]:
training_data = count_vector.fit_transform(X_train)
testing_data = count_vector.transform(X_test)

In [32]:
naive_bayes = MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)
naive_bayes.fit(training_data, y_train)
predictions = naive_bayes.predict(testing_data)
confusion_matrix(y_test, predictions)

array([[55, 73],
       [32, 73]])

In [33]:
print('Accuracy score: ', format(accuracy_score(y_test, predictions)))
print('Precision score: ', format(precision_score(y_test, predictions)))
print('Recall score: ', format(recall_score(y_test, predictions)))
print('F1 score: ', format(f1_score(y_test, predictions)))

Accuracy score:  0.5493562231759657
Precision score:  0.5
Recall score:  0.6952380952380952
F1 score:  0.5816733067729084


In [34]:
feature_coef = pd.concat([pd.DataFrame(count_vector.get_feature_names())
                          ,pd.DataFrame(naive_bayes.coef_[0])
                          ,pd.DataFrame(naive_bayes.feature_log_prob_[0])],axis=1)

In [35]:
feature_coef.columns = ['feature','coef','feature_log_prob']

In [36]:
feature_coef.sort_values(by='feature_log_prob',ascending=False)[:50]

,feature,coef,feature_log_prob
106,and,-3.567733,-3.577549
797,in,-3.848348,-3.762471
1522,the,-3.944363,-3.818711
1548,to,-3.793540,-3.914021
1699,years,-4.304365,-4.440114
1027,my,-4.282386,-4.487367
1073,of,-4.361524,-4.589150
338,computer,-4.486687,-4.625517
1326,science,-4.527509,-4.743300
639,for,-4.709830,-4.807839
